In [8]:
import pandas as pd


In [17]:
import nltk
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet


def remove_repeated_characters(tokens):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word != old_word else new_word
            
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens


def stopwords_removal(text):
    wh_words = ['my', 'your', 'his', 'her', 'this', 'do', "don't"]
    stop = set(stopwords.words('english'))
    for word in wh_words:
        stop.remove(word)
    tweet = ' '.join([x for x in text.split() if x not in stop])
    return tweet


def lemmatize(corpus):
    lem = WordNetLemmatizer()
    #corpus = [lem.lemmatize(x, pos = 'v') for x in corpus]
    corpus = [lem.lemmatize(x, pos = 'r') for x in corpus]
    corpus = [lem.lemmatize(x, pos = 'n') for x in corpus]
    return corpus


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
import nltk
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet


def remove_repeated_characters(tokens):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word != old_word else new_word
            
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens


def stopwords_removal(text):
    wh_words = ['my', 'your', 'his', 'her', 'this', 'do', "don't"]
    stop = set(stopwords.words('english'))
    for word in wh_words:
        stop.remove(word)
    tweet = ' '.join([x for x in text.split() if x not in stop])
    return tweet


def lemmatize(corpus):
    lem = WordNetLemmatizer()
    #corpus = [lem.lemmatize(x, pos = 'v') for x in corpus]
    corpus = [lem.lemmatize(x, pos = 'r') for x in corpus]
    corpus = [lem.lemmatize(x, pos = 'n') for x in corpus]
    return corpus


def stem(corpus, stem_type = None):
    stemmer = SnowballStemmer(language='english')
    corpus = [stemmer.stem(x) for x in corpus]
    return corpus


def pre_processing(tweet):
    """
    input: a string
    output: tokenized and preprocessed text (a list)
    """

    tweet = tweet.lower()

    #changing urls with a unique word
    p1 = re.compile(r'\bhttps://t.co/\w+')
    p2 = re.compile(r'\bhttps://^(t.co/\w+)\w')
    tweet = re.sub(p1, 'tweeturl', tweet)
    tweet = re.sub(p2, 'simpleurl', tweet)

    #changing all spaces with a simple space and dots and commas
    p3 = re.compile(r'\s+')
    p4 = re.compile(r"[.,;'’-]")
    tweet = re.sub(p4, '', tweet)
    tweet = re.sub(p3, ' ', tweet)


    #removing stopwords
    tweet = stopwords_removal(tweet)

    #tokenize the tweet
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    tweet = tokenizer.tokenize(tweet)
    tweet = remove_repeated_characters(tweet)

    #lemmatization and stemming
    tweet = lemmatize(tweet)
    tweet = stem(tweet)
    
    return tweet


def bag_of_words(tweets):
    """
    input:  a series of tokenized tweets (lists)
    output: a vocabulary (list) and a bag of words (numpy array)
    """

    set_of_words = set()
    for tweet in tweets:
        for word in tweet:
            set_of_words.add(word)
    vocab = list(set_of_words)

    position = {}
    for i, token in enumerate(vocab):
        position[token] = i

    bow_matrix = np.zeros((len(tweets), len(vocab)))

    for i, preprocessed_sentence in enumerate(tweets):
        for token in preprocessed_sentence:   
            bow_matrix[i][position[token]] = bow_matrix[i][position[token]] + 1

    return vocab, bow_matrix


def oversampler(dict_of_words, noise):
    """
    input: dict of words is a dictionary key: part of sentence - value: a dataframe of tokens and their relative frequency
            noise is a series of relative frequencies, indexed by tokens
    output: a random string
    """

    adj = np.random.choice(dict_of_words['a'].tokens, 1, p=dict_of_words['a'].freq)
    noun = np.random.choice(dict_of_words['n'].tokens, 3, p=dict_of_words['n'].freq)
    verb = np.random.choice(dict_of_words['v'].tokens, 2, p=dict_of_words['v'].freq)
    extra = np.random.choice(dict_of_words['r'].tokens, 3, p=dict_of_words['r'].freq)
    noise = np.random.choice(noise.index, 3, p=noise.values)

    
    return ' '.join([noun[0],extra[0],verb[0],noun[1],extra[1],adj[0],noun[2],verb[1],extra[2]\
        ,noise[0],noise[1],noise[2]]
        )

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
df = pd.read_csv('Documents/data/tweet_df_class.csv', index_col='Datetime').drop('Unnamed: 0', axis=1)
tweets_text = df.original_text
prep_tweets = tweets_text.apply(pre_processing)

Creating two datasets: one of "malaria tweets" and one of "not malaria tweets"

In [13]:
df['preproc'] = prep_tweets
nodup_df = df.drop_duplicates(subset='original_text')
tmp_tweets = nodup_df[['preproc', 'class']].reset_index(drop=True)
cases_tweets = tmp_tweets[tmp_tweets['class']==1]
not_cases_tweets = tmp_tweets[tmp_tweets['class']==0]

Creating a series with the most frequently used tokens in "malaria tweets" and their frequencies (bag of words is used)

In [14]:
vocab, bow_matrix = bag_of_words(cases_tweets.preproc)
bow_df = pd.DataFrame(bow_matrix, columns = vocab)
common_words = bow_df.sum(axis=0).sort_values()[-286:]

Creating the dataframe with oversampled tweets

In [15]:
#reading the labeled common words dataframe
words_df = pd.read_csv('Documents/data/common_words.csv')

#creating a dictionary with key = part of sentence and value = dataframe of tokens related to that part of sentence
#a is adjective, n is noun, v is verb and r is "other"
words_split = {}
for i, words in words_df.groupby('part'):
    words.freq = words.freq/sum(words.freq) #you may change words.freq with the frequency of only test data tokens
    words_split.update({str(i) : words.drop('part', axis=1)})

#absolute frequencies of each category
ss = words_df.groupby('part').sum()

#creating a series of uncommon words (index) and their relative frequencies 
noise_words = bow_df.sum(axis=0).sort_values()[:-286] #here too you can use noises from only test data tokens
noise_words /= sum(noise_words)

#generating fake tweets from the oversampling function

def fake_tweetter(n):

    fake_tweets = pd.DataFrame(np.array([oversampler(words_split, noise_words) for i in range(n)]).T,
            columns = ['preproc'])
    fake_tweets.preproc = fake_tweets.preproc.str.split()
    fake_tweets.insert(1, 'class', 1)

    return fake_tweets

#tweets with oversampling
z = 1000
oversampled_tweets = pd.concat([tmp_tweets, fake_tweetter(z)]).reset_index(drop=True)

Splitting the dataset

In [16]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score

# slitting variables with target and dependent variables
X = oversampled_tweets['preproc']
y = oversampled_tweets['class']

#bag of words
vocab, X = bag_of_words(X)

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X[:-z], y[:-z], test_size=0.3)

#oversampling
fakes = np.c_[X[-z:], y[-z:]]
i_fakes = np.random.choice(range(z), len(y_train[y_train==0])-len(y_train[y_train==1]), replace=False)
r_fakes = fakes[i_fakes]
X_train = np.r_[X_train, r_fakes[:, :-1]]
y_train = np.r_[y_train, r_fakes[:, -1]]